In [1]:
import os
import sys
os.chdir('../')
sys.path.append(os.path.join(os.getcwd(), "src"))

In [2]:
from dataclasses import dataclass
from pathlib import Path
from WattPredictor.utils.helpers import *
from WattPredictor.utils.exception import *
from WattPredictor.constants import *
from WattPredictor.utils.logging import logger

In [3]:
import sys
from datetime import datetime, timedelta, timezone
import pandas as pd
from WattPredictor.utils.logging import logger
from WattPredictor.utils.exception import CustomException
from WattPredictor.utils.feature import feature_store_instance


class MonitorDataLoader:
    def __init__(self, predictions_fg_name, predictions_fg_version, actuals_fg_name, actuals_fg_version):
        self.feature_store = feature_store_instance()
        self.predictions_fg_name = predictions_fg_name
        self.predictions_fg_version = predictions_fg_version
        self.actuals_fg_name = actuals_fg_name
        self.actuals_fg_version = actuals_fg_version

    def load_predictions_and_actuals(self, from_date: datetime, to_date: datetime) -> pd.DataFrame:
        try:
            from_date = from_date.astimezone(timezone.utc)
            to_date = to_date.astimezone(timezone.utc)

            predictions_fg = self.feature_store.feature_store.get_feature_group(
                name=self.predictions_fg_name,
                version=self.predictions_fg_version
            )

            actuals_fg = self.feature_store.feature_store.get_feature_group(
                name=self.actuals_fg_name,
                version=self.actuals_fg_version
            )

            predictions_df = predictions_fg.read()
            actuals_df = actuals_fg.read()

            combined_df = pd.merge(
                predictions_df,
                actuals_df[['sub_region_code', 'date', 'demand']],
                on=['sub_region_code', 'date'],
                suffixes=('', '_actual')
            )

            # Filter combined_df between from_date and to_date
            mask = (combined_df['date'] >= from_date) & (combined_df['date'] <= to_date)
            filtered_df = combined_df.loc[mask].sort_values(by=['date', 'sub_region_code'])

            logger.info(f"Monitoring data prepared with {len(filtered_df)} records between {from_date} and {to_date}.")
            return filtered_df

        except Exception as e:
            logger.error("Failed to load monitoring data.")
            raise CustomException(e, sys)


In [4]:
if __name__ == "__main__":
    loader = MonitorDataLoader(
        predictions_fg_name='elec_wx_predictions',
        predictions_fg_version=2,
        actuals_fg_name='elec_wx_features',
        actuals_fg_version=2
    )

    from_date = datetime(2025, 4, 1, 0, 0, 0)
    to_date = datetime(2025, 5, 31, 23, 59, 59)

    monitoring_data = loader.load_predictions_and_actuals(from_date, to_date)

   

[2025-07-17 19:58:14,552: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-17 19:58:14,560: INFO: helpers: yaml file: config_file\params.yaml loaded successfully]
[2025-07-17 19:58:14,564: INFO: helpers: yaml file: config_file\schema.yaml loaded successfully]
[2025-07-17 19:58:14,567: INFO: external: Initializing external client]
[2025-07-17 19:58:14,569: INFO: external: Base URL: https://c.app.hopsworks.ai:443]
[2025-07-17 19:58:17,649: INFO: python: Python Engine initialized.]

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1240214
[2025-07-17 19:58:20,377: INFO: feature_store: Connected to Hopsworks Feature Store: WattPredictor]
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.50s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.44s) 
[2025-07-17 19:58:29,927: INFO: 3727724080: Monitoring data prepared with 11 records between 2025-03-31 18:30:00+00:00 and 2025-05-31

In [5]:
monitoring_data

,sub_region_code,predicted_demand,date,demand
6,0,1616.0,2025-05-20 00:00:00+00:00,1735
1,1,744.0,2025-05-20 00:00:00+00:00,983
4,2,1297.0,2025-05-20 00:00:00+00:00,1577
8,3,548.0,2025-05-20 00:00:00+00:00,609
3,4,651.0,2025-05-20 00:00:00+00:00,845
7,5,1248.0,2025-05-20 00:00:00+00:00,1297
0,6,985.0,2025-05-20 00:00:00+00:00,1069
5,7,290.0,2025-05-20 00:00:00+00:00,292
10,8,641.0,2025-05-20 00:00:00+00:00,619
2,9,5648.0,2025-05-20 00:00:00+00:00,5485
